In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_project').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv('/FileStore/tables/Ecommerce_Customers.csv', header = True,
                     inferSchema = True)

In [5]:
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

Out[ 12 ]: 
['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
assembler = VectorAssembler(inputCols= ['Avg Session Length', 
                                        'Time on App', 
                                        'Time on Website', 
                                        'Length of Membership'],
                           outputCol = 'features')

In [9]:
output = assembler.transform(data)

In [10]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [11]:
# take a look at the output data
output.head(1)

Out[ 21 ]: [Row(Email=u'mstephenson@fernandez.com', Address=u'835 Frank TunnelWrightmouth, MI 82180-9605', Avatar=u'Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [12]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [13]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [14]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 326|
 mean| 498.5322321284077|
 stddev| 80.06877708921174|
 min| 256.67058229005585|
 max| 744.2218671047146|
+-------+-------------------+

In [15]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 174|
 mean| 500.77880146801374|
 stddev| 78.0905034879686|
 min| 275.9184206503857|
 max| 765.5184619388373|
+-------+-------------------+

In [16]:
#linear regression model
lr = LinearRegression(featuresCol= 'features',
                     labelCol = 'Yearly Amount Spent')

In [17]:
lr_model = lr.fit(train_data)

In [18]:
test_results = lr_model.evaluate(test_data)

In [19]:
# the difference between test data and predicted data
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 12.060269426695413|
 1.2880798759073855|
 5.594932884722425|
 -12.0647141559827|
-3.1834937580510996|
 22.761750004104442|
 1.471330505969945|
 3.5395099020994962|
 -6.713491672422265|
-3.4284796302929976|
 18.72650793412913|
 17.627690211947822|
 -4.026126219304672|
-25.834016298386018|
 3.0223335834180602|
-10.698115835135013|
-17.366379136324724|
 -8.76857456851775|
-1.5183409989782604|
 12.072728731200527|
+-------------------+
only showing top 20 rows

In [20]:
test_results.rootMeanSquaredError

Out[ 32 ]: 10.706502094184122

In [21]:
test_results.r2

Out[ 33 ]: 0.9810939016914563

In [22]:
real_data = test_data.select('features')

In [23]:
predictions = lr_model.transform(real_data)

In [24]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...|396.58008164593207|
[30.5743636841713...|440.77633388215827|
[31.0472221394875...|386.90246630429897|
[31.0662181616375...|460.99800736365705|
[31.2681042107507...| 426.654026931875|
[31.2834474760581...| 569.019339421563|
[31.3895854806643...|408.59828055401294|
[31.4459724827577...| 481.3374550330291|
[31.4474464941278...| 425.3162337676463|
[31.5171218025062...| 279.3469002806787|
[31.6005122003032...| 460.4463435569678|
[31.6098395733896...|426.91785943916034|
[31.6253601348306...|380.36302697622887|
[31.6739155032749...| 501.5590842082672|
[31.7366356860502...|493.91111267211386|
[31.8093003166791...| 547.4700151979762|
[31.8164283341993...| 518.4888706399811|
[31.8279790554652...|448.77132211545927|
[31.8627411090001...| 557.816482173025|
[31.9096268275227...| 551.3733069420387|
+--------------------+------------------+
only showing top 20 rows